In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import keras_tuner as kt

In [16]:
X = pd.read_csv('C:/Users/mandy/OneDrive/2022 IRS/ML/Datasets/Chinese Word Segmentation/4. Chinese Word Segmentation/as_training.utf8', names=['data'])
print(X.head())

                    data
0                   時間　：
1  三月　十日　（　星期四　）　上午　十時　。
2                   地點　：
3      學術　活動　中心　一樓　簡報室　。
4                   主講　：


In [17]:
y = X.copy()
X = X['data'].str.replace('\u3000', '')
print(X.head())
print(y.head())

0               時間：
1    三月十日（星期四）上午十時。
2               地點：
3      學術活動中心一樓簡報室。
4               主講：
Name: data, dtype: object
                    data
0                   時間　：
1  三月　十日　（　星期四　）　上午　十時　。
2                   地點　：
3      學術　活動　中心　一樓　簡報室　。
4                   主講　：


In [21]:
# labels = [[] * len(y)]
# for index, line in enumerate(y.data):
#     for char_index in range(len(line) - 1):
#         if char_index == len(line) - 1:
#             labels[index].append(1)
#         elif line[char_index + 1] == '\u3000':
#             labels[index].append(1)
#         else:
#             labels[index].append(0)

# print(labels)

IndexError: list index out of range

In [ ]:
def create_labels(data):
    label = []
    for index in range(len(data)):
        if index == len(data) - 1:
            label.append(1)
        elif data[index + 1] == '\u3000':
            label.append(1)

y.data.apply(lambda )